# TitleAbstract Reviewer

In this notebook, we will demostrate the use of `TitleAbstractReviewer` agents.

## Setting up the notebook

High-level configs

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [13]:
import pandas as pd 

from lattereview.providers import LiteLLMProvider
from lattereview.agents import TitleAbstractReviewer
from lattereview.workflows import ReviewWorkflow

## Data

First, let's load a dummy dataset of some articles that we once pooled for a prior systematic review published [here](https://pubmed.ncbi.nlm.nih.gov/36292201/).

In [3]:
data = pd.read_csv('data.csv')
data

,title,abstract,DOI,study_type,clinical_application,organ,modality,cardiovascular,task,deep_learning,external_validation
0,(18)F-FDG PET/CT Uptake Classification in Lymp...,Background Fluorine 18 ((18)F)-fluorodeoxygluc...,10.1148/radiol.2019191114,cross-sectional,diagnosis,lung,pet,0,classification,1,0
1,(18)F-FDG-PET/CT Whole-Body Imaging Lung Tumor...,Under the background of (18)F-FDG-PET/CT multi...,10.1155/2021/8865237,cross-sectional,diagnosis,lung,pet,0,classification,1,0
2,3-D Convolutional Neural Networks for Automati...,Deep two-dimensional (2-D) convolutional neura...,10.1109/jbhi.2018.2879449,cross-sectional,diagnosis,lung,ct,0,detection,1,0
3,3D CNN with Visual Insights for Early Detectio...,The 3D convolutional neural network is able to...,10.1155/2021/6695518,cross-sectional,diagnosis,lung,ct,0,classification,1,0
4,3D deep learning based classification of pulmo...,Classifying ground-glass lung nodules (GGNs) i...,10.1016/j.compmedimag.2020.101814,cross-sectional,diagnosis,lung,ct,0,"segmentation, classification",1,0
...,...,...,...,...,...,...,...,...,...,...,...
973,Vulture-Based AdaBoost-Feedforward Neural Fram...,"In today's scenario, many scientists and medic...",10.1007/s12539-022-00505-3,cross-sectional,diagnosis,lung,xr,0,"segmentation, classification",1,0
974,Wavelet decomposition facilitates training on ...,The adoption of low-dose computed tomography (...,10.1007/s00418-020-01961-y,cross-sectional,diagnosis,lung,ct,0,classification,1,0
975,Weakly unsupervised conditional generative adv...,Because of the rapid spread and wide range of ...,10.1016/j.media.2021.102159,cross-sectional,diagnosis,lung,ct,0,classification,1,0
976,Weakly-supervised lesion analysis with a CNN-b...,Objective.Lesions of COVID-19 can be clearly v...,10.1088/1361-6560/ac4316,cross-sectional,diagnosis,lung,ct,0,classification,1,0


Let's then write some inclusion and exclusion criteria to screen these papers.

In [4]:
inclusion_criteria = "The study must involve CT scans. If multiple modalities are involved, CT scans should be among them."
exclusion_criteria = "The study must not include PET scans as one of its modalities."

## Review

Then, we will define three reviewers (two juniors and one senior) that will go through the dataset and look for those items that meet all the inclusion criteria and does not violate any of the exclusion criteria. Each agent return either of the following scores:

- 1 means absolutely to exclude.
- 2 means better to exclude.
- 3 Not sure if to include or exclude.
- 4 means better to include.
- 5 means absolutely to include.

In [8]:
Agent1 = TitleAbstractReviewer(
    provider=LiteLLMProvider(model="gemini/gemini-1.5-flash"),
    name="Agent1",
    backstory="a PhD researcher in biology and computer science",
    inclusion_criteria = inclusion_criteria,
    exclusion_criteria = exclusion_criteria,        
    max_concurrent_requests=30, 
    model_args={"max_tokens": 200, "temperature": 0.1},
)

Agent2 = TitleAbstractReviewer(
    provider=LiteLLMProvider(model="gpt-4o-mini"),
    name="Agent2",
    backstory="a PhD researcher in biology and computer science",
    inclusion_criteria = inclusion_criteria,
    exclusion_criteria = exclusion_criteria,        
    max_concurrent_requests=30, 
    model_args={"max_tokens": 200, "temperature": 0.1},
)

Agent3 = TitleAbstractReviewer(
    provider=LiteLLMProvider(model="gpt-4o"),
    name="Agent3",
    backstory="a senior MD-PhD researcher with years of experience in conducting systematic reviews in radiology and deep learning",
    inclusion_criteria = inclusion_criteria,
    exclusion_criteria = exclusion_criteria,        
    max_concurrent_requests=30, 
    model_args={"max_tokens": 200, "temperature": 0.1},
    additional_context="""
    Two PhD reviewers have already reviewed this article and disagree on how to evaluate it. You can read their evaluation above.
    """
)

Finally, let's create a review workflow and put our agents to work!

In [10]:
def filter_func(row):
    score1 = int(row["round-A_Agent1_output"]["evaluation"])
    score2 = int(row["round-A_Agent2_output"]["evaluation"])
    if score1 != score2:
        if score1 >= 4 and score2 >= 4:
            return False
        if score1 >= 3 or score2 >= 3:
            return True
    elif score1 == score2 == 3:
        return True
    return False

async def review(df, sample_size=None):
    
    title_abs_review = ReviewWorkflow(
        workflow_schema=[
                {
                    "round": 'A',
                    "reviewers": [Agent1, Agent2],
                    "text_inputs": ["title", "abstract"]
                },
                {
                    "round": 'B',
                    "reviewers": [Agent3],
                    "text_inputs": ["title", "abstract", "round-A_Agent1_output", "round-A_Agent2_output"],
                    "filter": filter_func
                }
            ]
        )

    if sample_size:
        df = df.sample(sample_size)

    updated_df = await title_abs_review(df)
    total_cost = title_abs_review.get_total_cost()
    print(f"\n====== Finished reviewing ======\n")
    print(f"\nTotal cost: {total_cost}")
    print("-" * 100)
    return updated_df


updated_df = asyncio.run(review(data, sample_size=10))


====== Starting review round A (1/2) ======

Processing 10 eligible rows


['round: A', 'reviewer_name: Agent1'] -                     2025-02-01 10:32:15: 100%|██████████| 10/10 [00:00<00:00, 10.74it/s]


The following columns are present in the dataframe at the end of Agent1's reivew in round A: ['title', 'abstract', 'DOI', 'study_type', 'clinical_application', 'organ', 'modality', 'cardiovascular', 'task', 'deep_learning', 'external_validation', 'round-A_Agent1_output', 'round-A_Agent1_reasoning', 'round-A_Agent1_evaluation']


['round: A', 'reviewer_name: Agent2'] -                     2025-02-01 10:32:16: 100%|██████████| 10/10 [00:01<00:00,  8.89it/s]


The following columns are present in the dataframe at the end of Agent2's reivew in round A: ['title', 'abstract', 'DOI', 'study_type', 'clinical_application', 'organ', 'modality', 'cardiovascular', 'task', 'deep_learning', 'external_validation', 'round-A_Agent1_output', 'round-A_Agent1_reasoning', 'round-A_Agent1_evaluation', 'round-A_Agent2_output', 'round-A_Agent2_reasoning', 'round-A_Agent2_evaluation']

====== Starting review round B (2/2) ======

Processing 1 eligible rows


['round: B', 'reviewer_name: Agent3'] -                     2025-02-01 10:32:17: 100%|██████████| 1/1 [00:02<00:00,  2.15s/it]

The following columns are present in the dataframe at the end of Agent3's reivew in round B: ['title', 'abstract', 'DOI', 'study_type', 'clinical_application', 'organ', 'modality', 'cardiovascular', 'task', 'deep_learning', 'external_validation', 'round-A_Agent1_output', 'round-A_Agent1_reasoning', 'round-A_Agent1_evaluation', 'round-A_Agent2_output', 'round-A_Agent2_reasoning', 'round-A_Agent2_evaluation', 'round-B_Agent3_output', 'round-B_Agent3_reasoning', 'round-B_Agent3_evaluation']

====== Finished reviewing ======


Total cost: 0.0026831500000000005
----------------------------------------------------------------------------------------------------


In [11]:
updated_df.head()

,title,abstract,DOI,study_type,clinical_application,organ,modality,cardiovascular,task,deep_learning,external_validation,round-A_Agent1_output,round-A_Agent1_reasoning,round-A_Agent1_evaluation,round-A_Agent2_output,round-A_Agent2_reasoning,round-A_Agent2_evaluation,round-B_Agent3_output,round-B_Agent3_reasoning,round-B_Agent3_evaluation
162,Artificial Intelligence-assisted chest X-ray a...,OBJECTIVES: To study whether a trained convolu...,10.1007/s00330-020-07628-5,cross-sectional,diagnosis,lung,xr,0,classification,1,0,"{'evaluation': 1, 'reasoning': 'The study uses...","The study uses chest X-rays, not CT scans, thu...",1,{'reasoning': 'The study focuses solely on che...,The study focuses solely on chest X-rays (CXR)...,1,None,None,None
432,Deep-learning algorithms for the interpretatio...,The recent medical applications of deep-learni...,10.1371/journal.pone.0242759,retrospective cohort,diagnosis,lung,xr,0,classification,1,0,"{'evaluation': 2, 'reasoning': 'The study does...",The study does not involve CT scans; therefore...,2,{'reasoning': 'The study does not involve CT s...,The study does not involve CT scans as a prima...,1,None,None,None
137,An original deep learning model using limited ...,OBJECTIVES: Artificial intelligence (AI) has b...,10.1002/mp.15549,cross-sectional,diagnosis,lung,ct,0,classification,1,1,"{'evaluation': 4, 'reasoning': 'The study uses...",The study uses CT scans and does not mention P...,4,{'reasoning': 'The study involves CT scans and...,The study involves CT scans and does not menti...,5,None,None,None
724,Next-Generation Radiogenomics Sequencing for P...,PURPOSE: Considerable progress has been made i...,10.1007/s11307-020-01487-8,cross-sectional,diagnosis,lung,ct,0,classification,0,0,"{'evaluation': 1, 'reasoning': 'The study incl...","The study includes PET scans, violating the ex...",1,{'reasoning': 'The study involves CT scans as ...,The study involves CT scans as part of its mul...,1,None,None,None
867,Segmentation of infected region in CT images o...,"Since the outbreak of COVID-19 in 2019, the ra...",10.1038/s41598-021-01502-0,cross-sectional,diagnosis,lung,ct,0,segmentation,1,0,"{'evaluation': 1, 'reasoning': 'The abstract m...",The abstract mentions using 63 cases of MSD lu...,1,{'reasoning': 'The study involves CT scans for...,The study involves CT scans for segmenting inf...,5,{'reasoning': 'The abstract clearly states tha...,The abstract clearly states that the study inv...,5
